In [ ]:
# 라이브러리 install
# !pip install numpy==1.16.5 protobuf==3.11.0 grpcio==1.24.3 h5py==2.9.0 keras==2.2.4 absl-py==0.7.0 google-pasta==0.1.8 tensorflow-estimator==2.1.0 termcolor==1.1.0
# !pip install jupyter
# !pip install tensorflow
# !pip install metplotlib
# !pip install scikit-learn

# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다. 
import tensorflow as tf
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
# 텐서플로우 및 데이터 불러오기
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# 훈련/테스트 데이터 형태 확인
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# 10개 훈련 데이터 이미지 시각화
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 10, figsize=(10,10))

for i in range(10):
    axs[i].imshow(X_train[i], cmap='gray_r')
    axs[i].axis('off')
plt.show()

In [ ]:
# 10개 데이터 답 확인
[y_train[i] for i in range(10)]

In [ ]:
# 넘파이 불러오기
import numpy as np

# 종류 및 종류별 수 확인
np.unique(y_train, return_counts=True)

### 로지스틱 회귀 진행

In [ ]:
# 데이터 정규화 진행
# 픽셀 사이즈가 0~255사이 정수값이므로 255로 나누어 0~1사이로 정규화 진행
X_train_scaled = X_train / 255.0
X_train_scaled.shape

In [ ]:
# 넘파일 배열로 변환
X_train_scaled = X_train_scaled.reshape(-1, 28*28)
X_train_scaled.shape

In [ ]:
X_train_scaled[0]

In [ ]:
X_train[0]

In [ ]:
# SGDClassifier 사용하여 교차검증
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier

sgdc = SGDClassifier(loss='log', max_iter=5, random_state=42)  # loss='log_loss'
score = cross_validate(sgdc, X_train_scaled, y_train, n_jobs=-1)

print(np.mean(score['test_score']))

In [ ]:
sgdc = SGDClassifier(loss='log', max_iter=20, random_state=42)  # loss='log_loss'
score = cross_validate(sgdc, X_train_scaled, y_train, n_jobs=-1)

print(np.mean(score['test_score']))

In [ ]:
# 텐서플로 불러오기
import tensorflow as tf
from tensorflow import keras

In [ ]:
from sklearn.model_selection import train_test_split

X_train_scaled, X_test_val_scaled, y_train, y_test_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

X_train_scaled.shape, y_train.shape

In [ ]:
X_test_val_scaled.shape, y_test_val.shape

In [ ]:
# 레이어를 조립할 수 있는 공간, Qequential
from tensorflow.keras.models import Sequential

model = keras.Sequential()  # 모델 생성

In [ ]:
# 밀집층(dense layer) 생성
# 출력층 생성
from tensorflow.keras.layers import Dense

dense = Dense(10, activation='softmax')
model.add(dense)  # 모델에 출력층 추가

In [ ]:
# 입력층(input layer) 생성
from tensorflow.keras.layers import Input

input_layer = Input(shape=(784,))  # 입력층 생성
model.add(input_layer)  # 모델에 입력층 추가

In [ ]:
# compile 조립하여 완료작업
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 케라스 이진분류시손실함수 loss='binary_crossentropy'
# 케라스 다중분류시손실함수 loss='categorical_crossentropy'
# 정수 타겟값을 이용하여 크로스엔트로피 손실을 계산하는 경우 loss='sparse_categorical_crossentropy'
# 데이터 타겟값이 원-핫인코딩으로 되어 있있다면 loss='categorical_crossentropy'

In [ ]:
y_train[:10]

In [ ]:
# 모델 훈련
model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
# 성능 평가
model.evaluate(X_test_val_scaled, y_test_val)

In [ ]:
for i in range(10):
    
    y_pred = model.predict(X_train_scaled[i:i+1])
    print(f'실제 : {y_train[i]}', end=',')

    for j in range(10):
        if y_pred[0][j] == y_pred.max():
            print(f' 예측 :[{j}]')


### 심층 신경망

In [1]:
# 데이터 불러오기
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 데이터 스케일링
X_train_scaled = X_train / 255  # 픽셀이 0~255라서
X_train_scaled = X_train_scaled.reshape(-1, 28*28)

# 데이터 나누기
from sklearn.model_selection import train_test_split
X_train_scaled, X_val_scaled, y_train, y_test = train_test_split(X_train_scaled, y_train,
                                                                  test_size=0.2, random_state=42)

In [2]:
# 모델 공간 만들기
from tensorflow.keras.models import Sequential
model = keras.Sequential()

# 입력층 생성
input_layer = keras.Input(shape=(784,))

# 출력층 생성
dense1 = keras.layers.Dense(100, activation='sigmoid')
dense2 = keras.layers.Dense(10, activation='softmax')

# 모델에 더하기
model.add(input_layer)
model.add(dense1)
model.add(dense2)

In [3]:
# 레이어 정보 확인
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [4]:
# dense_1층의 모델 파라미터 개수
dense_1 = 784 * 100 + 100
dense_1

78500

In [5]:
# dense_2층의 모델 파라미터 개수
dense_2 = 100 * 10 + 10
dense_2

1010

In [6]:
# total 파라미터 개수
dense_1 + dense_2

79510

In [7]:
# 층을 추가하는 다른 방법
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='fashion Mnist model')

In [8]:
model.summary()

Model: "fashion Mnist model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 권장되는 방식
model = keras.Sequential()

model.add(keras.layers.Input(shape=(784,)))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 정보 확인
# model.summary()

# 모델 확정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습 진행
model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5717 - accuracy: 0.8038
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4133 - accuracy: 0.8502
Epoch 3/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3775 - accuracy: 0.8627
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3542 - accuracy: 0.8708
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3371 - accuracy: 0.8774


In [10]:
# Input layer 대신 Dense layer로 수행
model = keras.Sequential()

model.add(keras.layers.Dense(200, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 정보 확인
# model.summary()

# 모델 확정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6113 - accuracy: 0.7876
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4138 - accuracy: 0.8489
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3742 - accuracy: 0.8650
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3485 - accuracy: 0.8735
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3301 - accuracy: 0.8784


In [11]:
# 입력층 다음에 Flatten layer 추가
# 입력-Flatten-Dense-출력 layer
model = keras.Sequential()

# 입력-Flatten-Dense-출력 layer
model.add(keras.layers.Input(shape=(784,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 정보 확인
# model.summary()

# 모델 확정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5683 - accuracy: 0.8064
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4127 - accuracy: 0.8520
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3775 - accuracy: 0.8633
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3537 - accuracy: 0.8710
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3377 - accuracy: 0.8775


In [23]:
X_train_scaled2 = X_train_scaled.reshape(-1, 28, 28)
X_train_scaled2.shape

(48000, 28, 28)

In [24]:
# Input층 shape형태 변경
model = keras.Sequential()

model.add(keras.layers.Input(shape=(28,28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

# model.summary()

# 모델 확정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled2, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5671 - accuracy: 0.8062
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4113 - accuracy: 0.8523
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3764 - accuracy: 0.8649
Epoch 4/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3531 - accuracy: 0.8718
Epoch 5/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3360 - accuracy: 0.8784


In [25]:
# Flatten 층에 input 포함하여 진행
model = keras.Sequential()

# Flatten+input층-Dense-출력층
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 정보 확인
# model.summary()

# 모델 확정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled2, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5664 - accuracy: 0.8086
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.4100 - accuracy: 0.8519
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3749 - accuracy: 0.8637
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3537 - accuracy: 0.8723
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3360 - accuracy: 0.8802


In [28]:
# sigmoid 함수 대신, Relu 함수 적용
model = keras.Sequential()

model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# model.summary()

# 모델 확정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled2, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5265 - accuracy: 0.8128
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3885 - accuracy: 0.8616
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3533 - accuracy: 0.8722
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3328 - accuracy: 0.8802
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3171 - accuracy: 0.8865


In [34]:
# 검증 set 성능 확인

X_val_scaled2 = X_val_scaled.reshape(-1, 28, 28)
# X_val_scaled2.shape

model.evaluate(X_val_scaled2, y_test)

375/375 [==============================] - 1s 2ms/step - loss: 0.3406 - accuracy: 0.8828


[0.34064197540283203, 0.8828333616256714]